In [ ]:
import pandas as pd

# Load the datasets
prices_df = pd.read_excel('Processed_EURUSD_prices_Hourly.xlsx')
news_df = pd.read_excel('forex_sentiment_analysisnolabel_modified.xlsx')

# Convert 'Date' columns to datetime format
news_df['Date'] = pd.to_datetime(news_df['Date'].astype(str).str.replace('Z', '', regex=False), errors='coerce', utc=True).dt.tz_convert(None)
prices_df['Date'] = pd.to_datetime(prices_df['Date'], errors='coerce')

# Group news_df by date and sum the sentiment scores
news_grouped = news_df.groupby(news_df['Date'].dt.date)['sentiment'].sum().reset_index()

# Function to classify sentiment based on the sum of sentiment scores
def classify_sentiment(score):
    if score > 0:
        return 1
    elif score < 0:
        return -1
    else:
        return 0

# Apply the sentiment classification to the grouped data
news_grouped['sentiment'] = news_grouped['sentiment'].apply(classify_sentiment)

# Merge the datasets on the date
aligned_df_corrected = pd.merge(prices_df, news_grouped, left_on=prices_df['Date'].dt.date, right_on='Date', how='left')

# Forward fill the sentiment values to handle missing data
aligned_df_corrected['sentiment'] = aligned_df_corrected['sentiment'].fillna(method='ffill')

# Drop unnecessary columns if created during merging
aligned_df_corrected.drop(columns=['Date_y'], inplace=True, errors='ignore')
aligned_df_corrected.rename(columns={'Date_x': 'Date'}, inplace=True)

# Save the corrected aligned dataset to an Excel file
aligned_df_corrected.to_excel('aligned_forex_sentiment_data_corrected.xlsx', index=False)

print("The corrected aligned Forex sentiment dataset has been successfully saved as 'aligned_forex_sentiment_data_corrected.xlsx'.")
